<a href="https://colab.research.google.com/github/viniciusrpb/116319_estruturasdedados/blob/main/cap9_2_first_cnns.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Capítulo 9

### 9.2. Criando Redes Neurais Convolucionais


In [83]:
from keras.models import Sequential
from keras.layers import Dense,MaxPooling2D,Activation,Dropout,Flatten,Conv2D
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import cv2
import matplotlib.pyplot as plt

Carregando o conjunto de dados de folhas que utilizaremos como exemplo:

### 9.2.2. Utilizando nossos próprios dados




In [75]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [76]:
!cp -r "/content/drive/My Drive/leafs" "leafs"

In [149]:
train_datagen = ImageDataGenerator(rescale=1./255.,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest',
                                   validation_split=0.2)

test_datagen = ImageDataGenerator(rescale=1./255)

train_set = train_datagen.flow_from_directory('leafs/leafs/train/',
                                        batch_size=32,
                                        class_mode='categorical',
                                        target_size=(128,128),
                                        subset='training')

validation_set = train_datagen.flow_from_directory('leafs/leafs/train/',
                                        batch_size=32,
                                        shuffle=False,
                                        class_mode='categorical',
                                        target_size=(128,128),
                                        subset='validation')

test_set = test_datagen.flow_from_directory('leafs/leafs/test/',
                                        batch_size=32,
                                        shuffle=False,
                                        class_mode='categorical',
                                        target_size=(128,128))

Found 160 images belonging to 4 classes.
Found 40 images belonging to 4 classes.
Found 100 images belonging to 4 classes.


In [130]:
number_filters = 5
kernel_dimensions = (5,5)
tam_strides = (1,1)
number_of_neurons = 10
num_classes = 4

depth = 3

In [131]:
model = Sequential()

model.add(Conv2D(filters=number_filters,kernel_size=kernel_dimensions,strides=tam_strides, padding='same',kernel_initializer='glorot_normal',activation='relu',input_shape=(64,64,depth)))

model.add(GlobalMaxPool2D())
# fullly connected faz o stretch do activation map 2d para um vetor
model.add(Flatten())
model.add(Dense(number_of_neurons))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))


In [145]:
model2 = Sequential()

model2.add(Conv2D(filters=64,kernel_size=(3,3),strides=tam_strides, padding='same',kernel_initializer='glorot_normal',activation='relu',input_shape=(128,128,depth)))
model2.add(MaxPooling2D(pool_size=(2, 2)))

model2.add(Conv2D(filters=32,kernel_size=(3,3),padding='same',kernel_initializer='glorot_normal',activation='relu'))
model2.add(MaxPooling2D(pool_size=(2, 2)))

model2.add(Conv2D(filters=16,kernel_size=(5,5),padding='same',kernel_initializer='glorot_normal'))
model2.add(Activation('relu'))
model2.add(MaxPooling2D(pool_size=(2, 2)))

# fullly connected faz o stretch do activation map 2d para um vetor
model2.add(Flatten())
model2.add(Dense(number_of_neurons))
model2.add(Activation('relu'))
model2.add(Dropout(0.5))
model2.add(Dense(num_classes))
model2.add(Activation('softmax'))

In [146]:
model2.summary()

Model: "sequential_33"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_61 (Conv2D)           (None, 128, 128, 64)      1792      
_________________________________________________________________
max_pooling2d_34 (MaxPooling (None, 64, 64, 64)        0         
_________________________________________________________________
conv2d_62 (Conv2D)           (None, 64, 64, 32)        18464     
_________________________________________________________________
max_pooling2d_35 (MaxPooling (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_63 (Conv2D)           (None, 32, 32, 16)        12816     
_________________________________________________________________
activation_59 (Activation)   (None, 32, 32, 16)        0         
_________________________________________________________________
max_pooling2d_36 (MaxPooling (None, 16, 16, 16)      

In [147]:
sgd = SGD(learning_rate=0.1)

model2.compile(loss='categorical_crossentropy',optimizer=sgd,metrics=['accuracy'])

In [150]:
model2.fit(train_set,validation_data=validation_set,epochs=20)

Epoch 1/20
5/5 [==============================] - 15s 3s/step - loss: 1.5241 - accuracy: 0.1875 - val_loss: 1.3861 - val_accuracy: 0.2750
Epoch 2/20
5/5 [==============================] - 12s 3s/step - loss: 1.3879 - accuracy: 0.2062 - val_loss: 1.3855 - val_accuracy: 0.2500
Epoch 3/20
5/5 [==============================] - 12s 3s/step - loss: 1.3876 - accuracy: 0.2188 - val_loss: 1.3850 - val_accuracy: 0.2750
Epoch 4/20
5/5 [==============================] - 12s 3s/step - loss: 1.3872 - accuracy: 0.1937 - val_loss: 1.3843 - val_accuracy: 0.2250
Epoch 5/20
5/5 [==============================] - 13s 3s/step - loss: 1.3844 - accuracy: 0.3000 - val_loss: 1.3801 - val_accuracy: 0.2750
Epoch 6/20
5/5 [==============================] - 12s 3s/step - loss: 1.3893 - accuracy: 0.1937 - val_loss: 1.3865 - val_accuracy: 0.2500
Epoch 7/20
5/5 [==============================] - 12s 3s/step - loss: 1.3853 - accuracy: 0.2188 - val_loss: 1.3851 - val_accuracy: 0.1500
Epoch 8/20
5/5 [==================

In [151]:
from sklearn.metrics import classification_report, confusion_matrix

Y_pred = model.predict(test_set, 100 // 32+1)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(test_set.classes, y_pred))
print('Classification Report')
target_names = ['Leaf1', 'Leaf6', 'Leaf9','Leaf12']
print(classification_report(test_set.classes, y_pred, target_names=target_names))

Confusion Matrix
[[24  0  1  0]
 [25  0  0  0]
 [25  0  0  0]
 [25  0  0  0]]
Classification Report
              precision    recall  f1-score   support

       Leaf1       0.24      0.96      0.39        25
       Leaf6       0.00      0.00      0.00        25
       Leaf9       0.00      0.00      0.00        25
      Leaf12       0.00      0.00      0.00        25

    accuracy                           0.24       100
   macro avg       0.06      0.24      0.10       100
weighted avg       0.06      0.24      0.10       100



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Jargões

Stride: pulo da evolução no processo de convolução entre uma imagem e um fitlro. O ideal é utilizarmos stride 1 ou 2 (caso a imagem seja grande). No stride 1, cada pixel da imagem é considerado na convolução, enquanto que no stride 2, o filtro é passado sobre a imagem a cada 2 pixels.

Stride horizontal Stride vertical